In [ ]:
%matplotlib inline

#
# import utilitaires
#
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

import helpers as hlp
import helpers.dataset.PlantVillage as pv
import helpers.features as feat


#
# parametres dataset PlantVillage
#
config_overrides = hlp.MetaObject.from_json("config_overrides.json")
executor = hlp.create_thread_pool_executor()

pv_config = pv.Config(executor)
if not config_overrides is None:
    hlp.MetaObject.override_from_object(pv_config, 
                                        config_overrides.dataset)

print("pv_config")
display(vars(pv_config))


#
# obtenir le dataset PlantVillage
#
pv_dataset = pv.load(pv_config)
pv_dataframe = pv_dataset.dataframe

print("PlantVillage", pv_dataframe.shape)

In [ ]:
#
# get some images
#
# pv_image_indices = [random.randint(0, pv_dataframe.shape[0]) for _ in range(12)]
pv_image_indices = [46764, 21155, 18039, 7483, 6065, 39987, 841, 52365, 46308, 55725, 58270, 60308]
pv_image_labels = pv_dataframe.loc[pv_image_indices, "label"].values
pv_images = [pv_dataset.get_image(i) for i in pv_image_indices]

print(pv_image_indices)
print(pv_image_labels)

In [ ]:
_, axes = plt.subplots(3, 4, figsize=(12, 3 * 4))
for ax, image, label in zip(axes.flatten(), pv_images, pv_image_labels):
    plt.sca(ax)
    label = label.replace("_", "\n")
    plt.title(label)
    plt.imshow(image)

plt.suptitle("ORB")
plt.tight_layout()
plt.show()